In [41]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

sys.path.insert(1, '/Users/nickkeesey/Desktop/learning-dynamics-behavior/src/utils')
import behavior_utils as bu

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### 

In [26]:
df = pd.read_csv('/Users/nickkeesey/Desktop/learning-dynamics-behavior/data/new_bonsai_df_cleaned.csv')

df = bu.add_session_column(df)

df.sort_values(by='subject_id', inplace=True)

df

,subject_id,session_date,nwb_suffix,rig,user_name,experiment_description,task,notes,session_start_time,session_end_time,...,reaction_time_mean,early_lick_rate,invalid_lick_ratio,double_dipping_rate_finished_trials,double_dipping_rate_finished_reward_trials,double_dipping_rate_finished_noreward_trials,lick_consistency_mean_finished_trials,lick_consistency_mean_finished_reward_trials,lick_consistency_mean_finished_noreward_trials,new_session
0,668463,2023-07-07,1,Tower-1,Lucas,NaN,Coupled Baiting,NaN,2023-07-07 14:52:24.298459-07:00,2023-07-07 16:14:04.346112,...,0.931917,0.470000,0.721908,0.092308,0.117647,0.070922,0.966332,0.964094,0.968220,1
1,668463,2023-07-10,0,Tower-1,Lucas,NaN,Coupled Baiting,NaN,2023-07-10 15:02:43.617225-07:00,2023-07-10 16:56:00.735798,...,0.257906,0.533333,0.759481,0.392617,0.450000,0.363636,0.844578,0.860236,0.836670,2
2,668551,2023-06-15,3,Tower-5,Lucas,NaN,Coupled Without Baiting,NaN,2023-06-15 14:31:02.214445-07:00,2023-06-15 15:21:43.186666,...,1.383353,0.218978,0.226983,0.368984,0.476562,0.138889,0.838369,0.757762,0.946067,1
3,668551,2023-06-16,1,Tower-5,Lucas,NaN,Coupled Without Baiting,NaN,2023-06-16 14:06:18.098724-07:00,2023-06-16 14:55:32.172148,...,0.183997,0.573333,0.325333,0.799308,0.784452,0.529412,0.777771,0.772814,0.811625,2
4,671087,2023-07-06,0,Tower-1,Travis,NaN,Coupled Baiting,NaN,2023-07-06 13:03:36.430257-07:00,2023-07-06 13:52:45.128836,...,0.326036,0.606667,0.416410,0.526132,0.488038,0.628205,0.825732,0.877053,0.688217,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,776293,2025-01-08,111701,447-1-A,Ella Hilton,NaN,Coupled Baiting,NaN,2025-01-08 11:17:01.686715-08:00,2025-01-08 12:15:05.445164,...,0.736009,0.073333,0.610415,0.129555,0.141593,0.038462,0.924658,0.901735,0.980769,1
2037,776296,2025-01-07,121657,447-3-C,Avalon Amaya,NaN,Coupled Baiting,NaN,2025-01-07 12:16:57.371110-08:00,2025-01-07 13:32:01.428214,...,0.213954,0.196667,0.811455,0.037931,0.006803,0.069930,0.975390,0.994898,0.955336,2
2038,776296,2025-01-08,132251,447-3-C,Avalon Amaya,NaN,Coupled Baiting,NaN,2025-01-08 13:22:51.765949-08:00,2025-01-08 14:38:01.948262,...,0.132381,0.220000,0.761496,0.020202,0.005650,0.041322,0.989081,0.999188,0.974380,3
2036,776296,2025-01-06,122357,447-3-C,Avalon Amaya,NaN,Coupled Baiting,NaN,2025-01-06 12:23:57.056318-08:00,2025-01-06 13:29:03.887625,...,0.909316,0.046667,0.618026,0.080153,0.066667,0.099010,0.963531,0.973418,0.947772,1


In [10]:
df_original = pd.read_csv('/Users/nickkeesey/Desktop/learning-dynamics-behavior/data/comparison_bonsai_df.csv')

df_original.sort_values(by='subject_id', inplace=True)

df_original

,subject_id,session_date,nwb_suffix,session,rig,user_name,curriculum_name,curriculum_version,current_stage_actual,task,...,data_source,abs(bias_naive),abs(logistic_Su2022_bias),abs(logistic_Bari2019_bias),abs(logistic_Hattori2019_bias),abs(logistic_Miller2021_bias),weekday,avg_trial_length_in_seconds,water_day_total_last_session,water_after_session_last_session
6169,662914,2023-09-19,0,1.0,Blue-1,Katrina Nguyen,NaN,NaN,NaN,Uncoupled Without Baiting,...,AIND_training_447_bonsai,0.041885,0.050190,0.063603,0.063459,0.343473,2,8.112245,1.000,0.200
6093,662914,2023-10-06,0,9.0,Green-2,Ella Hilton,NaN,NaN,NaN,Uncoupled Without Baiting,...,AIND_training_447_bonsai,0.671687,0.545351,0.409615,0.415166,0.329638,5,7.713004,NaN,NaN
6164,662914,2023-09-21,0,2.0,Blue-1,Katrina Nguyen,NaN,NaN,NaN,Uncoupled Without Baiting,...,AIND_training_447_bonsai,0.049645,0.255776,0.194388,0.254168,0.029090,4,7.659574,1.000,0.000
6131,662914,2023-09-29,0,5.0,Blue-1,Katrina Nguyen,NaN,NaN,NaN,Uncoupled Without Baiting,...,AIND_training_447_bonsai,0.382609,0.556757,0.359746,0.382169,0.354122,5,7.681366,1.340,0.221
6125,662914,2023-10-03,1,6.0,Green-2,Ella Hilton,NaN,NaN,NaN,Uncoupled Without Baiting,...,AIND_training_447_bonsai,0.072165,0.110015,0.071359,0.067324,0.012978,2,7.917889,1.000,0.103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,776296,2025-01-06,122357,1.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_1_WARMUP,Coupled Baiting,...,AIND_training_447_bonsai,0.058824,0.020272,0.005877,0.021927,0.021165,1,6.890459,1.632,0.890
47,776296,2025-01-08,132251,3.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_2,Coupled Baiting,...,AIND_training_447_bonsai,0.024024,0.067681,0.059504,0.032865,0.052103,3,6.224066,1.513,0.990
15,776296,2025-01-10,131550,5.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_FINAL,Uncoupled Baiting,...,AIND_training_447_bonsai,0.281106,0.201647,0.166458,0.162098,0.172733,5,10.344828,NaN,NaN
65,776296,2025-01-07,121657,2.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_2,Coupled Baiting,...,AIND_training_447_bonsai,0.190332,0.377489,0.278989,0.296081,0.276343,2,6.484150,1.711,0.920


In [33]:
def calculate_session_proportion(df):
    """
    Calculate session proportion for each subject by dividing current session number 
    by total number of sessions for that subject.
    
    Args:
    df : pd.DataFrame
        DataFrame containing 'subject_id' and 'session' columns
        
    Returns
    pd.DataFrame
        Input DataFrame with new 'session_proportion' column added
    """
    # Get unique subjects
    subjects = df['subject_id'].unique()
    
    # For each subject
    for subject in subjects:
        # Get total sessions for this subject
        subject_df = df[df['subject_id'] == subject]
        total_sessions = len(subject_df)
        
        # Calculate proportion and assign to new column
        df.loc[df['subject_id'] == subject, 'session_proportion'] = \
            df.loc[df['subject_id'] == subject, 'session'] / total_sessions
            
    return df

df = calculate_session_proportion(df_original)

df


,subject_id,session_date,nwb_suffix,session,rig,user_name,curriculum_name,curriculum_version,current_stage_actual,task,...,abs(bias_naive),abs(logistic_Su2022_bias),abs(logistic_Bari2019_bias),abs(logistic_Hattori2019_bias),abs(logistic_Miller2021_bias),weekday,avg_trial_length_in_seconds,water_day_total_last_session,water_after_session_last_session,session_proportion
6169,662914,2023-09-19,0,1.0,Blue-1,Katrina Nguyen,NaN,NaN,NaN,Uncoupled Without Baiting,...,0.041885,0.050190,0.063603,0.063459,0.343473,2,8.112245,1.000,0.200,0.111111
6093,662914,2023-10-06,0,9.0,Green-2,Ella Hilton,NaN,NaN,NaN,Uncoupled Without Baiting,...,0.671687,0.545351,0.409615,0.415166,0.329638,5,7.713004,NaN,NaN,1.000000
6164,662914,2023-09-21,0,2.0,Blue-1,Katrina Nguyen,NaN,NaN,NaN,Uncoupled Without Baiting,...,0.049645,0.255776,0.194388,0.254168,0.029090,4,7.659574,1.000,0.000,0.222222
6131,662914,2023-09-29,0,5.0,Blue-1,Katrina Nguyen,NaN,NaN,NaN,Uncoupled Without Baiting,...,0.382609,0.556757,0.359746,0.382169,0.354122,5,7.681366,1.340,0.221,0.555556
6125,662914,2023-10-03,1,6.0,Green-2,Ella Hilton,NaN,NaN,NaN,Uncoupled Without Baiting,...,0.072165,0.110015,0.071359,0.067324,0.012978,2,7.917889,1.000,0.103,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,776296,2025-01-06,122357,1.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_1_WARMUP,Coupled Baiting,...,0.058824,0.020272,0.005877,0.021927,0.021165,1,6.890459,1.632,0.890,0.200000
47,776296,2025-01-08,132251,3.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_2,Coupled Baiting,...,0.024024,0.067681,0.059504,0.032865,0.052103,3,6.224066,1.513,0.990,0.600000
15,776296,2025-01-10,131550,5.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_FINAL,Uncoupled Baiting,...,0.281106,0.201647,0.166458,0.162098,0.172733,5,10.344828,NaN,NaN,1.000000
65,776296,2025-01-07,121657,2.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_2,Coupled Baiting,...,0.190332,0.377489,0.278989,0.296081,0.276343,2,6.484150,1.711,0.920,0.400000


In [34]:
df_original.columns.tolist()

['subject_id',
 'session_date',
 'nwb_suffix',
 'session',
 'rig',
 'user_name',
 'curriculum_name',
 'curriculum_version',
 'current_stage_actual',
 'task',
 'notes',
 'experiment_description',
 'session_start_time',
 'session_end_time',
 'session_run_time_in_min',
 'water_in_session_foraging',
 'water_in_session_manual',
 'water_in_session_total',
 'water_after_session',
 'water_day_total',
 'base_weight',
 'target_weight',
 'target_weight_ratio',
 'weight_after',
 'laser_1_calibration_power',
 'laser_2_calibration_power',
 'laser_1_target_areas',
 'laser_2_target_areas',
 'commit_ID',
 'repo_url',
 'current_branch',
 'weight_after_ratio',
 'p_reward_sum_mean',
 'p_reward_sum_std',
 'p_reward_sum_median',
 'p_reward_contrast_mean',
 'p_reware_contrast_median',
 'effective_block_length_mean',
 'effective_block_length_std',
 'effective_block_length_median',
 'effective_block_length_min',
 'effective_block_length_max',
 'duration_gocue_stop_mean',
 'duration_gocue_stop_std',
 'duration_

In [49]:
def clean_numeric_features(df, cols, nan_strategy='knn', target_col='session_proportion', min_samples = 2,  knn_neighbors=5, add_indicator=False):
    """
    Clean numeric features by imputing missing values using KNNImputer
    
    Args:
        df: pd.DataFrame
            Input DataFrame
        cols: list
            List of columns to clean
        nan_strategy: str
            Strategy for imputing missing values
        min_samples: int
            Minimum number of samples in each neighborhood for KNNImputer
        knn_neighbors: int
            Number of neighbors to use for KNNImputer
        add_indicator: bool
            If True, add indicator columns for missing values
            
    Returns:
        pd.DataFrame
            DataFrame with cleaned numeric features
    """

    # Remove target samples under threshold
    class_counts = df[target_col].value_counts()
    valid_classes = class_counts[class_counts >= min_samples].index
    df_clean = df[df[target_col].isin(valid_classes).copy()]

    new_columns = []

    # Handle infinite values
    for col in cols:
        df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)

    if add_indicator and df_clean[col].isna().any():
        indicator_col = f'{col}_is_missing'
        df_clean[indicator_col] = df_clean[col].isna().astype(int)
        new_columns.append(indicator_col)

    # drop strat
    if nan_strategy == 'drop':
        df_clean = df_clean.dropna(subset=cols)
    
    # knn strat
    elif nan_strategy == 'knn':
        imputer = KNNImputer(n_neighbors=knn_neighbors)
        df_clean[cols] = imputer.fit_transform(df_clean[cols])

    # median strat
    elif nan_strategy == 'median' or nan_strategy == 'indicator':
        for col in cols:
            median_val = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(median_val)

    # Handle extreme values after imputation
    for col in cols:
        Q1 = df_clean[col].quantile(0.01)
        Q3 = df_clean[col].quantile(0.99)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean[col] = df_clean[col].clip(lower_bound, upper_bound)
        
    return df_clean, new_columns

In [50]:
feature_cols = ['p_reward_sum_mean', 
                'effective_block_length_mean',
                'ignore_rate', 'reward_rate', 'finished_rate', 'double_dipping_rate_finished_trials',
                'double_dipping_rate_finished_reward_trials',
                'double_dipping_rate_finished_noreward_trials',
                'lick_consistency_mean_finished_trials',
                'lick_consistency_mean_finished_reward_trials',
                'lick_consistency_mean_finished_noreward_trials',
                'foraging_performance', 'foraging_eff',
                'early_lick_rate', 
                'invalid_lick_ratio', 'double_dipping_rate_finished_trials', 
                'lick_consistency_mean_finished_trials','reaction_time_mean'
                ]

cat_cols = ['task']

target_col = 'session_proportion'

In [55]:
clean_df, _ = clean_numeric_features(df_original, 
                                  cols = feature_cols, 
                                  nan_strategy='knn'
                                  )
clean_df

/var/folders/mr/k7gx9xps333170k7zsggl4tw0000gn/T/ipykernel_29574/2746804976.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
/var/folders/mr/k7gx9xps333170k7zsggl4tw0000gn/T/ipykernel_29574/2746804976.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[cols] = imputer.fit_transform(df_clean[cols])
/var/folders/mr/k7gx9xps333170k7zsggl4tw0000gn/T/ipykernel_29574/2746804976.py:62: SettingWithCopyWarning: 
A value is trying to be se

,subject_id,session_date,nwb_suffix,session,rig,user_name,curriculum_name,curriculum_version,current_stage_actual,task,...,abs(bias_naive),abs(logistic_Su2022_bias),abs(logistic_Bari2019_bias),abs(logistic_Hattori2019_bias),abs(logistic_Miller2021_bias),weekday,avg_trial_length_in_seconds,water_day_total_last_session,water_after_session_last_session,session_proportion
6169,662914,2023-09-19,0,1.0,Blue-1,Katrina Nguyen,NaN,NaN,NaN,Uncoupled Without Baiting,...,0.041885,0.050190,0.063603,0.063459,0.343473,2,8.112245,1.000,0.200,0.111111
6093,662914,2023-10-06,0,9.0,Green-2,Ella Hilton,NaN,NaN,NaN,Uncoupled Without Baiting,...,0.671687,0.545351,0.409615,0.415166,0.329638,5,7.713004,NaN,NaN,1.000000
6164,662914,2023-09-21,0,2.0,Blue-1,Katrina Nguyen,NaN,NaN,NaN,Uncoupled Without Baiting,...,0.049645,0.255776,0.194388,0.254168,0.029090,4,7.659574,1.000,0.000,0.222222
6131,662914,2023-09-29,0,5.0,Blue-1,Katrina Nguyen,NaN,NaN,NaN,Uncoupled Without Baiting,...,0.382609,0.556757,0.359746,0.382169,0.354122,5,7.681366,1.340,0.221,0.555556
6125,662914,2023-10-03,1,6.0,Green-2,Ella Hilton,NaN,NaN,NaN,Uncoupled Without Baiting,...,0.072165,0.110015,0.071359,0.067324,0.012978,2,7.917889,1.000,0.103,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,776296,2025-01-06,122357,1.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_1_WARMUP,Coupled Baiting,...,0.058824,0.020272,0.005877,0.021927,0.021165,1,6.890459,1.632,0.890,0.200000
47,776296,2025-01-08,132251,3.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_2,Coupled Baiting,...,0.024024,0.067681,0.059504,0.032865,0.052103,3,6.224066,1.513,0.990,0.600000
15,776296,2025-01-10,131550,5.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_FINAL,Uncoupled Baiting,...,0.281106,0.201647,0.166458,0.162098,0.172733,5,10.344828,NaN,NaN,1.000000
65,776296,2025-01-07,121657,2.0,447-3-C,Avalon Amaya,Uncoupled Baiting,2.3,STAGE_2,Coupled Baiting,...,0.190332,0.377489,0.278989,0.296081,0.276343,2,6.484150,1.711,0.920,0.400000


In [56]:
def preprocess_data(df, feature_cols, cat_cols, target_col, test_size = 0.2, random_state = 42):
    """
    Preprocess data for machine basic neural network classification model

    Args: 
    df : pd.DataFrame
        Input df with features and targets
    feature_cols : list
        Names of numerical features
    cat_cols : list
        Names of categorical columns to encode
    target_col : str
        Name of target column to classify in output layer
    test_size: float
        Proportion of data to use for testing

    Returns:
    dict:
        X_train, X_test : preprocessed feature 
        y_train, y_test : encoded target variables
        encoders : dict of fitted LabelEncoders for each categorical feature
        scaler : fitted StandardScaler for numerical features
    """

    df_copy = df.copy()

    # Initialize encoders and scaler
    encoders = {}
    scaler = StandardScaler()
    
    # Categorical column handling 
    encoded_cats = []
    for col in cat_cols:
        if col != target_col:
            encoders[col] = LabelEncoder()
            df_copy[f'{col}_encoded'] = encoders[col].fit_transform(df_copy[col])
            encoded_cats.append(f'{col}_encoded')
    
    # Combine categorical and numerical features
    final_features = feature_cols + encoded_cats
    
    # Scale numerical features
    X = df_copy[final_features].copy()
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=final_features)

    # Encode target variable
    target_encoder = LabelEncoder()
    y = target_encoder.fit_transform(df_copy[target_col])
    encoders['target'] = target_encoder

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y,
        test_size = test_size,
        random_state = random_state,
        stratify = y
    )

    return {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train, 
        'y_test': y_test,
        'encoders': encoders,
        'scaler': scaler
    }

In [57]:


processed_df = preprocess_data(
    df = clean_df,
    feature_cols = feature_cols,
    cat_cols = cat_cols,
    target_col = target_col
)

processed_df

{'X_train':       p_reward_sum_mean  effective_block_length_mean  ignore_rate  \
 5859          -0.001586                     0.606058    -0.635018   
 434           -1.333085                     0.693696    -0.543439   
 3216           0.433133                    -0.732561    -0.806947   
 5129          -0.756765                    -0.227528    -0.803562   
 1790           1.313372                    -0.747223     2.204158   
 ...                 ...                          ...          ...   
 4642           2.166730                    -0.734849    -0.664187   
 1048          -0.001586                     0.708061     0.196006   
 1555          -0.084805                     0.019566     0.087018   
 5815           0.684527                    -0.476380    -0.447631   
 5037           0.331967                    -0.646116    -0.770180   
 
       reward_rate  finished_rate  double_dipping_rate_finished_trials  \
 5859     0.513383       0.635018                            -0.833456   